In [1]:

from torch import nn
import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel, AutoTokenizer
from torch.optim import Adam
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
import requests
from bs4 import BeautifulSoup

In [2]:
!pip install transformers

     |████████████████████████████████| 3.3 MB 8.8 MB/s eta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tokenizers-0.9.4


In [2]:
torch.cuda.empty_cache()
pd.set_option('display.max_rows', None)

In [3]:
### Here we set the labels of the topics, in this case we have 7 labels for comments and 8 for articles
label_to_id = {"politik" : 0, "maßnahmen" : 1, "infektion" : 2, "impfung": 3, "lockdown": 4, "wirtschaft":5, "lockerung":6, "überblick": 7}
dataset_topic = pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleaned2check_article.csv", names=["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment", "Topic_comment", "Topic_article", "Comment", "Method"], encoding="utf-8")
dataset_topic = dataset_topic.loc[dataset_topic["Topic_article"].isin(label_to_id.keys())]
dataset_topic = dataset_topic[["Comment", "Topic_article"]]
dataset_topic = dataset_topic.loc[dataset_topic["Topic_article"].isin(label_to_id.keys())]
dataset_topic.columns = ["Comment", "Topic"]
dataset_topic["Comment"]
df_train, df_test = np.split(dataset_topic.sample(frac=1, random_state=77), [int(.8*len(dataset_topic))])


dataset_topic["Topic"].value_counts()
len(dataset_topic)
dataset_topic.reset_index(drop=True, inplace=True)

df_train.drop(df_train.loc[df_train["Comment"].isnull()].index, axis = 0, inplace = True)
# for x in df_train["Comment"]:
#     if type(x) != str:
#         print(type(x), x)
dataset_topic


,Comment,Topic
0,Corona-Lockdown-Lockerungen: Deutsche suchen w...,lockerung
1,Corona-News am 07.03.: Olaf Scholz verspricht ...,überblick
2,Corona-News am 18.3.: Länder setzen Öffnungssc...,überblick
3,Rezo kritisiert die Corona-Politik: »So ein Ha...,politik
4,EU-Kommission genehmigt Finanzhilfen für Air F...,wirtschaft
5,Christine Lambrecht will Geimpfte von Corona-B...,impfung
6,Corona-News am 14.02.: Daniel Günther kritisie...,überblick
7,Impfstoff gegen Coronavirus: Russland nimmt di...,impfung
8,Corona-Krise: Deutschlands Schlachthöfe sind z...,infektion
9,Corona-Regeln für private Feiern: Mehrheit der...,maßnahmen


In [47]:
label_to_id = {"politik" : 0, "maßnahmen" : 1, "infektion" : 2, "impfung": 3, "lockdown": 4, "wirtschaft":5, "lockerung":6, "überblick":7}
dataset_topic = pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleaned2check.csv", names=["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment", "Topic_comment", "Topic_article", "Comment", "Method"], encoding="ISO-8859-1")
dataset_topic = dataset_topic.loc[dataset_topic["Topic_article"].isin(label_to_id.keys())]
dataset_topic.reset_index(drop=True, inplace=True)
for x in range(len(dataset_topic)):
    print(x)

    request = requests.get("https://www.spiegel.de/wissenschaft/medizin/corona-news-am-samstag-die-wichtigsten-entwicklungen-zu-sars-cov-2-und-covid-19-a-" +dataset_topic.loc[int(x), "ID"])
    soup = BeautifulSoup(request.content, "html.parser")
    title = soup.find("title").text
    subtitle = soup.find("meta", property="og:description")["content"]

    dataset_topic.loc[x, "Comment"] = title + " " + subtitle

dataset_topic


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


,ID,Date,Time,Comment Level,Username,Opinion,Sentiment,Topic_comment,Topic_article,Comment,Method
0,40a7c181-1ab8-4d4b-bca5-bf8f4ace27ea,2020-04-30,06:59:00,0,Karsten-QGRAZB3WR,negative,negative,lockerung,lockerung,Corona-Lockdown-Lockerungen: Deutsche suchen w...,manual
1,4736f7ec-6396-4f5f-991b-1f83e5f5df66,2021-03-07,17:19:00,0,Hospitalitaet,positive,positive,infektion,überblick,Corona-News am 07.03.: Olaf Scholz verspricht ...,manual_al
2,5994b2b8-4e42-49a7-b82c-c78e084df7f7,2021-03-18,13:40:00,0,Karl-Heinz-lFYMd87Mg,negative,negative,infektion,überblick,Corona-News am 18.3.: Länder setzen Öffnungssc...,manual_al
3,aa90b76d-f535-43bc-81da-759ff0a1f610,06.04.2021,18:01:00,0,Thomas-opcCIf_Gg,negative,negative,lockerung,politik,Rezo kritisiert die Corona-Politik: »So ein Ha...,manual_al
4,a24ee383-2310-4166-ad8b-4250259ada77,2021-04-06,08:55:00,0,Georg-e2NHZx9Zg,negative,negative,wirtschaft,wirtschaft,EU-Kommission genehmigt Finanzhilfen für Air F...,manual_al
...,...,...,...,...,...,...,...,...,...,...,...
3197,0210360b-673b-46a4-bb95-4998e452f1a4,2020-04-01,05:31:00,0,Jan-6-zu_9rWg,negative,negative,wirtschaft,wirtschaft,Coronavirus: SPD-Chefin Saskia Esken fordert S...,manual_al
3198,014a5d6d-ee0f-4094-b41b-01ed82391721,2020-10-15,18:07:00,0,prologo,neutral,negative,maßnahmen,lockdown,"Neue Corona-Maßnahmen: Reichen die Beschlüsse,...",manual
3199,3c9cef10-a61a-429a-a5e4-271b2d2f34bd,2020-08-04,02:38:00,0,Christina-XMNOcI7GR,positive,neutral,demos,überblick,Coronavirus News am 4.8.: Novavax berichtet vo...,manual
3200,855c1ae5-aa3b-4497-bf62-a2b450564320,2021-03-18,16:23:00,0,M.-wz918EnMg,positive,neutral,impfung,impfung,Corona: Ema hält an Empfehlung von AstraZeneca...,manual


In [48]:
dataset_topic["Comment"][0]
dataset_topic.to_csv("annotated_data/annotated_data_with_users_and_al_cleaned2check_article.csv", encoding="utf-8", index=False, header=False)

In [4]:
data_text = []
data_labels = []
text_train = []
labels_train = []
text_test = np.array([])
labels_test = np.array([])


tokenizer = AutoTokenizer.from_pretrained("bert-base-german-cased")




label_to_id = {"politik" : 0, "maßnahmen" : 1, "infektion" : 2, "impfung": 3, "lockdown": 4, "wirtschaft":5, "lockerung":6, "überblick":7 }
id_to_label = {0:"politik", 1:"maßnahmen", 2:"infektion", 3: "impfung", 4: "lockdown", 5:"wirtschaft", 6:"lockerung", 7:"überblick"}

### Here we create a training set that can be used to compare across different sizes of training data



class Dataset(torch.utils.data.Dataset):

    def __init__(self,dataframe):
        self.labels = [label_to_id[label] for label in dataframe["Topic"] if label in ["politik", "maßnahmen", "infektion", "impfung", "lockdown", "wirtschaft", "lockerung", "überblick"]]
        self.texts = [tokenizer(txt, padding ="max_length", max_length = 512, truncation=True, return_tensors="pt") for txt in dataframe["Comment"]]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_texts, batch_labels
# ### Define test and train set
# text_test = text_train[split_index+1:]
# text_train = text_train[:split_index]
# labels_test = labels_train[split_index+1:]
# labels_train = labels_train[:split_index]


### Replace label as int
for idx, labels in enumerate(data_labels):
    data_labels[idx] = label_to_id[labels]


class ClassifierText(nn.Module):
    def __init__(self, dropout = 0.5):
        super(ClassifierText, self).__init__()

        self.bert = BertModel.from_pretrained("bert-base-german-cased")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 8)
        self.relu = nn.ReLU()
    
    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids = input_id, attention_mask = mask, return_dict = False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer




accuracies = []
hello = []
# folds = StratifiedKFold(n_splits=5)
# for train_index, test_index in folds.split(data_text, data_labels):
#     text_train, labels_train  = data_text[train_index], data_labels[train_index]
#     text_test, labels_test  = data_text[test_index], data_labels[test_index]

#     train_dict = {"texts": text_train, "labels" : labels_train}
#     test_dict = {"texts": text_test, "labels" : labels_test}

    # print([len(z) for z in [text_train, text_test]])


    ### Remove newline characters
    # for idx, text in enumerate(train_dict["texts"]):
    #     train_dict["texts"][idx] = text.rstrip("\n")

    # for idx2, text2 in enumerate(test_dict["texts"]):
    #     test_dict["texts"][idx2] = text2.rstrip("\n")
        
    # data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    
    # meine_dataset_train = Dataset.from_dict(train_dict)

    # mein_dataset_test = Dataset.from_dict(test_dict)

    # tokenized_dataset_train = meine_dataset_train.map(preprocess_function, batched= True)
    # tokenized_dataset_test = mein_dataset_test.map(preprocess_function, batched=True)


In [5]:
def train(model, train_data, learning_rate, epochs):

    train = Dataset(train_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=8, shuffle=True)


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):

        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            
            batch_loss = criterion(output, train_label.long())
            total_loss_train += batch_loss.item()
            
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
        
        total_acc_val = 0
        total_loss_val = 0

        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
            | Train Accuracy: {total_acc_train / len(train_data): .3f}'
        )    
                  
EPOCHS = 10
model = ClassifierText()
LR = 1e-6
train(model, df_train, LR, EPOCHS)
# for train_index, test_index in folds.split(data_text, data_labels):
#     text_train, labels_train  = data_text[train_index], data_labels[train_index]
#     text_test, labels_test  = data_text[test_index], data_labels[test_index]

#     train_dict = {"texts": text_train, "labels" : labels_train}
#     test_dict = {"texts": text_test, "labels" : labels_test}              
#     train(model, text_train, LR, EPOCHS)

  0%|          | 1/321 [00:01<10:39,  2.00s/it]


RuntimeError: CUDA out of memory. Tried to allocate 48.00 MiB (GPU 0; 11.93 GiB total capacity; 6.21 GiB already allocated; 23.25 MiB free; 6.26 GiB reserved in total by PyTorch)

In [33]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=8)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
df_test.drop(df_test.loc[df_test["Comment"].isnull()].index, axis = 0, inplace = True)
evaluate(model, df_test)



Test Accuracy:  0.726


In [79]:
torch.save(model.state_dict(),"Model_ArticleTextFull.pt")

In [7]:
model = ClassifierText()
model.load_state_dict(torch.load("Model_ArticleTextFull.pt"))
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = model.to(device)

In [8]:
geladenes_model = model
evaluate(geladenes_model, df_test)

NameError: name 'evaluate' is not defined

In [34]:
from numpy.ma.core import argmax
from transformers import TextClassificationPipeline

use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

if use_cuda:

    mein_model = geladenes_model.cuda()

correct_per_topic = {"politik" : 0, "maßnahmen" : 0, "infektion" : 0, "impfung": 0, "lockdown": 0, "wirtschaft":0, "lockerung":0, "überblick":0}
true = 0
with torch.no_grad():

  for _, sample in df_test.iterrows():
    input_ids = tokenizer(sample["Comment"], return_tensors="pt", truncation=True)["input_ids"].to(device)

    masks = tokenizer(sample["Comment"], return_tensors="pt", truncation=True)["attention_mask"].to(device)
    
    prediction = model( input_ids, masks)
 
    class_pred = np.argmax(prediction.cpu()).item()

    topic = label_to_id[sample["Topic"]]

    #print(class_pred, topic)
    
    
    if class_pred == topic:
      true+= 1
      correct_per_topic[id_to_label[topic]] += 1


print(true/len(df_test))

for topic in df_test["Topic"].unique():
  correct_per_topic[topic] /= df_test["Topic"].value_counts()[topic]


    # prediction = model(mask=masks, input_id=input_ids)

    # topic = np.argmax(prediction)

    # topic
print(correct_per_topic)
print(df_train["Topic"].value_counts())

0.7293354943273906
{'politik': 0.46296296296296297, 'maßnahmen': 0.47560975609756095, 'infektion': 0.8461538461538461, 'impfung': 0.7936507936507936, 'lockdown': 0.5161290322580645, 'wirtschaft': 0.9523809523809523, 'lockerung': 0.0625, 'überblick': 0.9540816326530612}
überblick     799
wirtschaft    290
maßnahmen     288
infektion     270
impfung       260
lockdown      234
politik       182
lockerung     125
Name: Topic, dtype: int64


In [43]:
check = pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleaned2check.csv",names=["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment", "topic_comment", "Topic_article", "Comment", "Method"], encoding="ISO-8859-1", header=None)
print(check["Topic_article"].value_counts())
check

überblick        999
maßnahmen        425
wirtschaft       378
infektion        353
impfung          330
lockdown         306
politik          247
lockerung        164
usa              134
reise            111
demos             92
virus             86
maskenpflicht     68
arbeit            66
tests             59
krankenhaus       38
sport             35
china             19
Name: Topic_article, dtype: int64


,ID,Date,Time,Comment Level,Username,Opinion,Sentiment,topic_comment,Topic_article,Comment,Method
0,40a7c181-1ab8-4d4b-bca5-bf8f4ace27ea,2020-04-30,06:59:00,0,Karsten-QGRAZB3WR,negative,negative,lockerung,lockerung,SPON macht mit bei der Hetze der Covid19 Leugn...,manual
1,4736f7ec-6396-4f5f-991b-1f83e5f5df66,2021-03-07,17:19:00,0,Hospitalitaet,positive,positive,infektion,überblick,+++ Britische Forscher infizieren Freiwillige ...,manual_al
2,5994b2b8-4e42-49a7-b82c-c78e084df7f7,2021-03-18,13:40:00,0,Karl-Heinz-lFYMd87Mg,negative,negative,infektion,überblick,"Braucht es noch einen weiteren Nachweis, dass ...",manual_al
3,15557a4c-0e90-412e-8bd5-03ae3f6a7854,13.02.2021,17:57:00,0,Glaubnichtalles,neutral,neutral,virus,virus,Es sollte die Frage gestellt werden. Was ist d...,manual_al
4,aecb14c3-12f5-4f76-aab8-c3620a709fa4,2020-12-22,10:18:00,0,Schmandter-JT6IwOpMR,negative,negative,virus,virus,Es wird bis in den Januar hinein Untersuchunge...,manual_al
...,...,...,...,...,...,...,...,...,...,...,...
3991,855c1ae5-aa3b-4497-bf62-a2b450564320,2021-03-18,16:23:00,0,M.-wz918EnMg,positive,neutral,impfung,impfung,Ich halte die Freigabe trotz allem für vertret...,manual
3992,3d7819e6-f105-4d41-a2c4-1d06b6f51dc1,2020-05-22,07:15:00,0,Rüdiger-GQduR_6WR,negative,neutral,china,china,"Ich hoffe, dass das bald ein Ende hat. China b...",manual_al
3993,2fbfe25d-a788-409a-844d-fb7bd80bcf44,2021-01-09,13:26:00,0,Coraggioso-Ir6WKaaMR,positive,negative,impfung,lockdown,"Merkel: ""Ein Virus, das uns alle trifft, lässt...",manual
3994,55863de1-bf74-49d1-95fe-08d2fbd51970,15.04.2021,11:50:00,0,Maria-_AnWpnTWg,neutral,negative,sport,sport,"Moin, schon komisch, dass Vereine in den unt...",manual_al


In [3]:
pd.set_option("max_rows", None)
meine_daten = pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleaned2x.csv", names=["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment", "topic_comment", "Topic_article", "Comment", "Method"], encoding="ISO-8859-1", header=None)
meine_daten.loc[meine_daten["Topic_article"] == "intensivstation", "Topic_article"] = "krankenhaus"
meine_daten.loc[meine_daten["Topic_article"] == "homeoffice", "Topic_article"] = "arbeit"
meine_daten.loc[meine_daten["Topic_article"] == "schule", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "reiseverbot", "Topic_article"] = "reise"
meine_daten.loc[meine_daten["Topic_article"] == "quarantäne", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "schulschließung", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "demo", "Topic_article"] = "demos"
meine_daten.loc[meine_daten["Topic_article"] == "warnapp", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "intensivstationen", "Topic_article"] = "krankenhaus"
meine_daten.loc[meine_daten["Topic_article"] == "lockerungen", "Topic_article"] = "lockerung"
meine_daten.loc[meine_daten["Topic_article"] == "konjunktur", "Topic_article"] = "wirtschaft"
meine_daten.loc[meine_daten["Topic_article"] == "schulschliessung", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "inzidenz", "Topic_article"] = "infektion"
meine_daten.loc[meine_daten["Topic_article"] == "kontaktbeschränkung", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "schnelltests", "Topic_article"] = "tests"
meine_daten.loc[meine_daten["Topic_article"] == "verschärfungen", "Topic_article"] = "maßnahmen"
meine_daten.loc[meine_daten["Topic_article"] == "intesivstation", "Topic_article"] = "krankenhaus"
meine_daten.loc[meine_daten["Topic_article"] == "maskenplficht", "Topic_article"] = "maskenpflicht"
meine_daten.loc[meine_daten["Topic_article"] == "fußball", "Topic_article"] = "sport"
meine_daten = meine_daten[meine_daten.Topic_article != "XXX"]
meine_daten.to_csv("annotated_data/annotated_data_with_users_and_al_cleaned2x.csv", header=None, index=None)
meine_daten["Topic_article"].value_counts()

überblick              995
maßnahmen              411
wirtschaft             362
infektion              342
impfung                328
lockdown               299
politik                242
lockerung              164
usa                    132
reise                  100
demos                   90
virus                   81
maskenpflicht           63
arbeit                  62
tests                   54
krankenhaus             34
sport                   33
china                   19
tönnies                  7
masken                   4
antikörper               3
raus                     3
tourismus                3
testpflicht              3
einreise                 3
intesivstationen         2
fussball                 2
staatshilfen             2
homeschooling            1
sterberate               1
who                      1
mobilität                1
gewalt                   1
grenzkontrollen          1
kontaksperre             1
steuersenkung            1
übersicht                1
s

In [8]:
import pandas as pd
volle_daten= pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleaned2check.csv", names=["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment", "topic_comment", "Topic_article", "Comment", "Method"], encoding="ISO-8859-1", header=None)
volle_daten.topic_comment.value_counts()

politik          755
maßnahmen        603
impfung          549
infektion        424
wirtschaft       372
lockdown         323
reise            127
usa              127
lockerung        124
demos            114
virus            105
maskenpflicht     95
tests             89
arbeit            77
krankenhaus       42
sport             37
china             33
Column8            1
Name: topic_comment, dtype: int64